In [1]:
import cv2 
import numpy as np
import face_recognition
import os 
from datetime import datetime
from imutils import paths
import pickle 
import pandas as pd 
from pathlib import Path

In [2]:
min_dev_allowed = 0.42
image_list = list(paths.list_images('images/'))
with open ('image_filepaths', 'rb') as f:
    my_image_list = pickle.loads(f.read())
if image_list == my_image_list['imgs']:
    with open('face_encodings', 'rb') as f:
        Record = pickle.loads(f.read())
else:
    print('''New images have been added, please wait while images are encoded.
    Get comfortable :), this will take a while, I'll probably go watch a movie if I were you''')
    my_image_list = {'imgs': image_list}
    with open('image_filepaths', 'wb') as f:
        f.write(pickle.dumps(my_image_list))
    known_encodings = []
    known_names = []
    for image in image_list:
        name = image.split('\\')[-2].split('/')[-1]
        img_current = cv2.imread(image)
        img_rgb = cv2.cvtColor(img_current,  cv2.COLOR_BGR2RGB)
        faces = face_recognition.face_locations(img_rgb)
        encodings = face_recognition.face_encodings(img_rgb, faces)
        for encoding in encodings:
            known_names.append(name)
            known_encodings.append(encoding)
    
    print('Encoding complete')
    
    Record = {'names': known_names, 'encodings': known_encodings}
    
    with open('face_encodings', 'wb') as f:
        f.write(pickle.dumps(Record))





In [1]:
#Creates a new attendance file for each day, includes a check for replication 
# def registerattendance(name):
#     now = datetime.now()
#     timeString = now.strftime('%H:%M:%S')
#     dateString = now.strftime("%d/%m/%Y")
#     filename = f'Attendance for {dateString}'
#     new_df = pd.DataFrame(data = [[name, timeString]], columns = ['Name', 'TimeIn'])
#     if os.path.exists(filename):
#         df = pd.read_csv(filename)
#         filter = df['Name'] == name
#         condition = df[filter].empty
#         if condition:
#             new_df.to_csv(filename, mode ='a', index=False, header=False )
#     else:
#         new_df.to_csv(filename, mode ='a', index=False, header=True) 

In [3]:


#IMPLEMENTATION FOR ATTENDANCE USING OS
# def registerattendance(name):
#     with open('Attendance_Register.csv','r+') as f:
#         Data_list = f.readlines()
#         name_list =[]
#         #print(Data_list)
#         for line in Data_list:
#             entry = line.split(',')
#             name_list.append(entry[0])
#         if name not in name_list:
#             now = datetime.now()
#             timeString = now.strftime('%H:%M:%S')
#             f.writelines(f'\n{name},{timeString}')

#IMPLEMENTATION FOR ATTENDANCE USING PANDAS
def registerattendance(name):
    now = datetime.now()
    timeString = now.strftime('%H:%M:%S')
    dateString = now.strftime("%d/%m/%Y")
    filename = 'RAIN_Attendance_Register.csv'
    new_df = pd.DataFrame(data = [[name, timeString, dateString]], columns = ['Name', 'TimeIn', 'Date'])
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        filter =df [(df['Name'] == name) & (df['Date'] == dateString)]
        if len(filter) == 0:
            new_df.to_csv(filename, mode ='a', index=False, header=False)
    else:
        new_df.to_csv(filename, mode ='a', index=False, header=True)
            
            





In [ ]:
#Attendance capturing with OpenCV 
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    img_small = cv2.resize(img,(0,0), None, 0.25, 0.25)
    img_small = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)
    
    facesinCurFrame = face_recognition.face_locations(img_small)
    encodesCurFrame = face_recognition.face_encodings(img_small, facesinCurFrame)
    
    for encodeFace, faceLoc in zip(encodesCurFrame, facesinCurFrame):
        matches = face_recognition.compare_faces(Record['encodings'], encodeFace)
        faceDis = face_recognition.face_distance(Record['encodings'], encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
        min_faceDis = faceDis[matchIndex]
        
        
        #min_dev_allowed has been declared at the beginning of the code
        if min_faceDis < min_dev_allowed:
            name = Record['names'][matchIndex].upper()
            print(name)
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x2,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6), cv2.FONT_HERSHEY_COMPLEX,1, (255, 255, 255), 2)
            registerattendance(name)
        else:
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x2,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,'FACE IS NOT KNOWN',(x1+6,y2-6), cv2.FONT_HERSHEY_COMPLEX,1, (255, 255, 255), 2)
            
        
    cv2.imshow('frame', img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

